In [2]:
from collections import defaultdict
import random
from multiprocessing.pool import ThreadPool
import time
import os


import Tree_simulator_fitting as cts
from Simulate_epidemic_fitting import *
from vector_comparisons import *
from movement_fitting import *


import tempfile
import pyabc

FileNotFoundError: [Errno 2] No such file or directory: '/localdisk/home/s1732989/ABM/Fitting/epidemic_size.csv'

In [3]:
import numpy as np

In [6]:
results_path = "Looping models/Results/Fitting/LTT/"

run_number = 2

try:
    os.mkdir(os.path.join(dropbox_path, results_path, str(run_number)))
except FileExistsError:
    pass

In [7]:
observed_SS = get_observed_SS()

print(observed_SS)

new_LTT = list(observed_SS[2])
      
observed = {"a":new_LTT, "b":observed_SS[7], "c":observed_SS[6]}

#This needs to be a dictionary not an array

#observed = new_LTT

(array([1.82881143e-01, 3.39365382e-02, 1.05112375e-02, 6.92756254e-03,
       2.76445567e-04, 1.29848969e-02, 9.08707574e-03, 3.40119680e-04,
       8.02417069e-03, 4.95533309e-03, 1.84276642e-04, 2.64530390e-01,
       2.33433201e-01, 2.31927608e-01]), array([8.63844135e-01, 1.35995419e-01, 1.26764264e-05, 1.38535232e-04,
       3.73410329e-07, 7.11397137e-06, 4.16021717e-08, 1.70519081e-06]), array([7.64716360e-02, 3.28948609e-05, 2.07757444e-01, 7.15588995e-01,
       1.49030353e-04]), array([0.04682632, 0.09794978, 0.10515936, 0.11696818, 0.11646749,
       0.07191204, 0.0550219 , 0.04590037, 0.03945282, 0.04368183,
       0.05707097, 0.05308446, 0.03243955, 0.03808636, 0.03168514,
       0.02121876, 0.01492135, 0.00901105, 0.00157196, 0.00157029]), [(0, 0.021361168777556623), (0.021361168777556623, 0.042722337555113246), (0.042722337555113246, 0.06408350633266988), (0.06408350633266988, 0.08544467511022649), (0.08544467511022649, 0.10680584388778311), (0.10680584388778311, 0.1281

In [8]:
def compare_LTT_stats(obs_LTT, coalescent_tree):
    
    sim_LTT_pre = LTT.calculate_LTT_metrics(coalescent_tree.lineages_through_time)
    sim_LTT = normalise(sim_LTT_pre)
    
    LTT_stat_difference = np.linalg.norm(obs_LTT - sim_LTT)
    
    return LTT_stat_difference



In [9]:
def distance(observed, sim):
    
    coalescent_tree = sim[0]
    dist = sim[1]
    ch = sim[2]
    
    observed_LTT = observed[0]
    obs_dist = observed[1]
    obs_ch = observed[2]
    
    a_dist = compare_LTT_stats(observed_LTT, coalescent_tree)
    b_dist = get_jumps(obs_dist, dist)
    c_dist = get_jumps(obs_ch, ch)
    
    distance = (a+b+c)/3
    
    #return distance
    return {"a":a_dist, "b":b_dist, "c":c_dist}

In [71]:
def distance2(x,y): #inputs are the dictionaries
    
    new_a_x = np.array(x['a'])
    new_a_y = np.array(y['a'])
    
    dist_a = np.linalg.norm(new_a_x - new_a_y)
    dist_b = np.linalg.norm(x["b"] - y["b"])
    dist_c = np.linalg.norm(x["c"] - y["c"])
    
    final_b = dist_b/y["b"]
    final_c = dist_c/y["c"]

    dist = dist_a + final_b + final_c
    
    return dist
    

In [72]:
dict1 = {"a":0.01,"b":21, "c":100}
dict2 = {"a":0.05,"b":33, "c":110}

distance2(dict1, dict2)


0.4945454545454545

In [34]:
distance = pyabc.PNormDistance(p=2)

In [16]:
def simulate_pyabc(parameter):
    result = simulate_epidemic(**parameter)
    return {"a":result[0], "b":result[1], "c":result[2]} #so this returns the sample as a dictionary

In [17]:
parameters = dict(a=(0.5,1), b=(0,0.3), c=(0,0.5))

In [40]:
#prior = pyabc.Distribution(a=pyabc.RV("uniform", 0.5, 1), b=pyabc.RV("uniform",0, 0.3), c=pyabc.RV("uniform", 0, 0.5))
#prior = pyabc.Distribution(a=pyabc.RV("uniform", 0.5, 1))

prior = pyabc.Distribution(**{key: pyabc.RV("uniform", a, b - a) for key, (a,b) in parameters.items()})

abc = pyabc.ABCSMC(simulate_pyabc, prior, distance2)

In [41]:
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

abc_id = abc.new(db_path, observed)

history = abc.run(max_nr_populations=10, minimum_epsilon=0.1)

INFO:History:Start <ABCSMC(id=31, start_time=2020-02-19 15:32:45.144119, end_time=None)>
INFO:ABC:Calibration sample before t=0.
Process Process-30:
  File "/Users/s1743989/Documents/GitHub/ABSynth/Fitting_modules/ABCSMC/epidemic_function_fitting.py", line 26, in run_epidemic
    assignment = focal_case.who_am_I(infected_individuals_set, popn_size, option_dict_districtlevel, contact_structure, case_dict, parent, day, cfr, distributions) #Assign the current case id to an individual
Process Process-35:
Process Process-34:
Process Process-31:
Process Process-25:
Process Process-32:
Process Process-29:
Process Process-26:
Process Process-28:
Process Process-36:
Process Process-33:
Process Process-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/s1743989/anaconda3/lib/python3.6/multiprocessing/process.py", line 25

KeyboardInterrupt: 

  File "/Users/s1743989/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/s1743989/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 491, in simulate_one
    t, theta, summary_statistics)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 491, in simulate_one
    t, theta, summary_statistics)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 491, in simulate_one
    t, theta, summary_statistics)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/sampler/multicore_evaluation_parallel.py", line 32, in work
    new_sim = simulate_one()
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 491, in simulate_one
    t, theta, summary_statistics)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/p

  File "/Users/s1743989/Documents/GitHub/ABSynth/Fitting_modules/ABCSMC/epidemic_function_fitting.py", line 26, in run_epidemic
    assignment = focal_case.who_am_I(infected_individuals_set, popn_size, option_dict_districtlevel, contact_structure, case_dict, parent, day, cfr, distributions) #Assign the current case id to an individual
  File "/Users/s1743989/Documents/GitHub/ABSynth/Fitting_modules/ABCSMC/Simulate_epidemic_fitting.py", line 114, in run_model
    day_dict, case_dict, nodes, trans_dict, child_dict, dist_mvmt, ch_mvmt, onset_times, epidemic_capped = run_epidemic(0, original_day_dict, susceptibles_left , original_case_dict, original_trans_dict, original_child_dict, infected_individuals_set, popn_size, option_dict_district_level, original_onset_times, original_nodes, cdf_len_set, cdf_array, original_dist_mvmt, original_ch_mvmt, contact_structure, cfr, distributions, iteration_count, capped, epidemic_length, case_limit, a, b, c)
Traceback (most recent call last):
  File "/Us

  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/smc.py", line 491, in simulate_one
    t, theta, summary_statistics)
KeyboardInterrupt
KeyboardInterrupt
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 270, in sample
    return self.sample_function(pars)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-packages/pyabc/model.py", line 121, in summary_statistics
    raw_data = self.sample(pars)
  File "/Users/s1743989/anaconda3/lib/python3.6/site-package

  File "/Users/s1743989/Documents/GitHub/ABSynth/Fitting_modules/ABCSMC/epidemic_function_fitting.py", line 26, in run_epidemic
    assignment = focal_case.who_am_I(infected_individuals_set, popn_size, option_dict_districtlevel, contact_structure, case_dict, parent, day, cfr, distributions) #Assign the current case id to an individual
  File "/Users/s1743989/Documents/GitHub/ABSynth/Fitting_modules/ABCSMC/Simulate_epidemic_fitting.py", line 114, in run_model
    day_dict, case_dict, nodes, trans_dict, child_dict, dist_mvmt, ch_mvmt, onset_times, epidemic_capped = run_epidemic(0, original_day_dict, susceptibles_left , original_case_dict, original_trans_dict, original_child_dict, infected_individuals_set, popn_size, option_dict_district_level, original_onset_times, original_nodes, cdf_len_set, cdf_array, original_dist_mvmt, original_ch_mvmt, contact_structure, cfr, distributions, iteration_count, capped, epidemic_length, case_limit, a, b, c)
  File "/Users/s1743989/Documents/GitHub/ABSyn

In [21]:
print(new_LTT)


[0.07647163596358295, 3.289486088152504e-05, 0.20775744412099748, 0.715588994701951, 0.00014903035258703715]


In [32]:
a=0.97
b=0.05 
c=0.15

for i in range(100):
    if i %10 == 0:
        print(i)
    result = simulate_epidemic(a,b,c)
    if result[0] != "W":
       
        print(type(result[0]), type(result[1]), type(result[2]))

0
10


/Users/s1743989/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if __name__ == '__main__':


<class 'numpy.ndarray'> <class 'int'> <class 'int'>
20
30
40


KeyboardInterrupt: 

In [52]:
c = 10
c2 = 15

np.linalg.norm(c2-c)

5.0

In [8]:
a = [1,2,3]

In [9]:
b = np.array(a)

In [10]:
print(len(b))

3


In [13]:
if b.size == 3:
    print('a')

a
